In [1]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

from tqdm import tqdm , trange
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sklearn.metrics import *

In [ ]:
import pandas as pd
!git clone https://gitlab.com/vibss2397/nlp.git
df = pd.read_csv("nlp/fce_final.csv")

Cloning into 'nlp'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.
Checking out files: 100% (7/7), done.


In [2]:
!ls nlp

# Old Data Processing

In [ ]:
df1 = df[['incorrect', 'correct', 'start_off', 'end_off', 'err_type']]
df1 = np.array(df1)

In [ ]:
for i in range(227, 242):
  df1[i][1] = 'In the twenty-first century, things are changing very fast. If we look at  family life, they have a lifestyle which is typical of these times. Children have their own televisions, computers and telephones in their room. Mother and Father have their own cars and mobile phones. Sometimes even the children have their own mobile phone as well. The house : fire alarm, burglar alarm, house temperature gauge, automatic garage door, etc. They use all of this because they want to do their work and leisure activities with these helping kits.' 

In [ ]:
final = []
for ele in df1:
  if(type(ele[0])!=float and type(ele[1])!=float):
    incorr = ele[0]
    corr = ele[1]
    # print(incorr+'\n', corr)
    corr = ele[1].split('. ')
    corr = [ele for ele in corr if len(ele)>0]
    corr = [ele if (ele[-1]=='.' or ele[-1]==',' or ele[-1]=='?' or ele[-1]=='!') else ele+'.' for ele in corr]
    incorr_arr = []
    
    start_off = ele[2]
    end_off = ele[3]
    temp = ''
    sent = 0
    counter = 0
    pos_start, pos_end = 0, 0
    for line in corr:
      last_word =  line.split(' ')[-1].strip('.').strip(',').strip('?') if len(line.split(' '))==1 else line.split(' ')[-2].strip('.').strip(',').strip('?')
      # last_word_final = last_word if last_word.find('.')<0 else last_word[:-1]
      incorr_find = incorr.find(last_word, pos_end)
      pos_end = incorr_find + len(last_)
      incorr_arr.append(incorr[pos_start:pos_end])
      pos_start = pos_end + 2
    final.append(tuple([incorr_arr, corr, start_off, end_off, ele[4]]))

In [ ]:
# pd.DataFrame(final, columns =['incorr', 'corr', 'start_off', 'end_off', 'err'])
final

[(['In the twenty first century things are changing very fast. ',
   'If we look at a family life. They have typical  lifestyle  typical  these times. ',
   'Children have  own televisions, computers and telephones in their room. ',
   'Mother and Father have  own cars and mobile phones. ',
   'Sometimes even the children have  own mobile phone as well. ',
   '',
   'he twenty first century things are changing very fast. If we look at a family life. They have typical  lifestyle  typical  these times. Children have  own televisions, computers and telephones in their room. Mother and Father have  own cars and mobile phones. Sometimes even the children have  own mobile phone as well. Full of technological tools and instruments . Fire alarm, burglar alarm, house temperature checker, automatic garage door many of them. They use all of this because they want to do their work, and leisure activities with this helping kits.'],
  ['In the twenty-first century, things are changing very fast.',
 

In [ ]:
for tup in final[1:]:
  incorr = tup[0]
  corr = tup[1]
  corr = [ele[:-1] if (ele[-1]=='.' or ele[-1]==',' or ele[-1]=='?' or ele[-1]=='!') else ele for ele in corr ]
  start = 0


['I have received your letter, which  makes me happy', 'For the first time in my life I have won a competition', 'I would like to take you up on your invitation in July', 'I am only able to take my holiday in July', 'The rest of the year I work']


In [ ]:
final[1]

(['I have received your letter which is make me happy',
  'The first time in my life I have won  competition',
  'I would like to attempt your invite in July',
  'I  only able to take my holidy in July,  rest of the year I work.'],
 ['I have received your letter, which  makes me happy',
  'For the first time in my life I have won a competition',
  'I would like to take you up on your invitation in July',
  'I am only able to take my holiday in July',
  'The rest of the year I work.'],
 27,
 27,
 'OT')

In [ ]:
!ls nlp

ls: cannot access 'nlp': No such file or directory


In [ ]:
df1 = df[['incorrect', 'err_type']]
df1 = df1.rename(columns={'incorrect':'input','err_type':'err'})
df2 = df[['correct']]
# df2['err'] = 'CO'
df2.insert(1, 'err', 'CO')
df2 = df2.rename(columns={'correct':'input','err':'err'})

In [ ]:
frames = [df1, df2]
result = pd.concat(frames)

In [ ]:
df3 = result.groupby(by='input')['err'].unique().apply(lambda x: ','.join(x)).reset_index()

In [ ]:
df3['err'] = df3.err.str.split(',')

In [ ]:
def remove_arr(arr, ele):
  if ele in arr:
    arr.remove(ele)
  return arr

df3['err'] = df3['err'].apply((lambda x: ['CO'] if 'CO' in x else x))
df3['err'] = df3['err'].apply(lambda x: remove_arr(x, 'DS')).apply(lambda x: remove_arr(x, 'SM'))

In [ ]:
df3_final = df3[df3['err'].apply(lambda x: len(x)!=0)]

In [ ]:
err_categories = sorted(['LX', 'OT', 'GM', 'CO'])

In [ ]:
for err_type in err_categories:
  df3_final[err_type] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def make_1(row):
  for ele in row[1]:
    row[2 + err_categories.index(ele)] = 1
  return row
df3_final = df3_final.apply(make_1, axis = 1)

In [ ]:
df3_final.head()

,input,err,CO,GM,LX,OT
0,"""She was very angry when I arrived. She stare...","[LX, GM, OT]",0,1,1,1
1,"""She was very angry when I arrived. She stare...",[CO],1,0,0,0
2,"100 years ago, people dressed differently. ...",[CO],1,0,0,0
3,"1st ""F"" is from ""fast-food"", something fast a...","[LX, OT]",0,0,1,1
4,2nd F is 'fulfill' wear ever you have under y...,"[LX, OT, GM]",0,1,1,1


In [ ]:
df4 = df3_final.drop('err', axis = 1)
# df4.to_csv('processed_fce.csv')

# New Data Processing

In [ ]:
df4 = pd.read_csv('nlp/fce_final.csv')
df4.head()

In [ ]:
#selecting sentence and labels
df4 = df4[['0', '1']]

In [ ]:
df4['len'] = df4.apply(lambda x: len(x[0].split('. ')), axis=1)
df5 = df4[['input', 'len']]
df5_numpy = df5.to_numpy()
max_len = ['', float('-inf')]
for ele in df5_numpy:
  if(ele[1]>max_len[1]):
    max_len[0] = ele[0]
    max_len[1] = ele[1]
df4 = df4.drop('len', axis=1)
final_df = pd.concat([df4['input'], df4.apply(lambda x: np.array(x[1:], dtype=np.float), axis=1)], axis = 1)

In [3]:
max_len

In [ ]:
train_data = final_df.sample(frac=0.80, random_state=0)
test_data = final_df.drop(train_data.index)
train_data_arr = train_data.to_numpy()
test_data_arr = test_data.to_numpy()

print("Train shape", train_data_arr.shape)
print("Test shape", test_data_arr.shape)

Train shape (14266, 2)
Test shape (3567, 2)


In [ ]:
train_data.head(5)

,input,0
16165,"Unfortunately, Pat wasn't very good at keeping...","[0.0, 0.0, 1.0, 0.0]"
7256,I hope you will take my proposal into favourab...,"[1.0, 0.0, 0.0, 0.0]"
4397,"Firstly, I must admit that when we realised th...","[0.0, 0.0, 1.0, 0.0]"
1053,Also the clothes in a hundred years will have ...,"[1.0, 0.0, 0.0, 0.0]"
17340,With your agreement we could change the progra...,"[0.0, 0.0, 1.0, 0.0]"


In [ ]:
weights = np.array(train_data[0].agg('sum'), dtype=np.float)
weights_sum = np.sum(weights)
weights2 = weights_sum/weights
weights2 = weights2 / weights2[0]
# weights2 = np.exp(weights2)/sum(np.exp(weights2))
print(weights2)

[1.         1.70293486 1.15317499 2.08136483]


In [ ]:
# Final Training

In [ ]:
class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
          self.bert = BertModel.from_pretrained("bert-base-uncased")
          for name, param in self.bert.named_parameters():
            param.requires_grad = True
          # add your additional layers here, for example a dropout layer followed by a linear classification head
          self.dropout = nn.Dropout(0.5)
          self.out1 = nn.Linear(768, 4)
          self.relu = nn.ReLU()
          self.out2 = nn.Linear(256, 4)

    def forward(self, input_ids, token_type_ids, attention_mask):
          bert_outputs = self.bert(
               input_ids, 
               attention_mask,
               token_type_ids
          )
          out1 = self.out1(bert_outputs[1])
          # out1 = self.relu(out1)
          # out1 = self.dropout(out1)
          # logits = self.out2(out1)
          return out1

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bert_model = CustomBERTModel().to(device)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, max_len):
        self.data = data
        self.targets = targets
        self.max_len = max_len
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        x = self.data[index]
        label = torch.tensor(self.targets[index], dtype=torch.float)
        return [x, label]
    def __len__(self):
        return len(self.data)

In [ ]:
def encode_sentence(sentence, max_len):
    return bert_tokenizer.batch_encode_plus(
                        sentence,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        padding = 'max_length',
                        truncation = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt'    # Return pytorch tensors.
                   )
train_dataset = MyDataset(train_data['input'].to_numpy(), train_data[0].to_numpy(), max_len[1])

validation_dataset = MyDataset(test_data['input'].to_numpy()[:len(test_data['input'])//2], 
                               test_data[0].to_numpy()[:len(test_data['input'])//2], max_len[1])

test_dataset = MyDataset(test_data['input'].to_numpy()[len(test_data['input'])//2:], 
                               test_data[0].to_numpy()[len(test_data['input'])//2:], max_len[1])

In [ ]:
bs = 16
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            shuffle = True,
            # sampler = SequentialSampler(train_dataset), # Select batches randomly
            batch_size = bs # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            validation_dataset,  # The training samples.
            sampler = SequentialSampler(validation_dataset), # Select batches sequentially
            batch_size = bs # Trains with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset,  # The training samples.
            sampler = SequentialSampler(test_dataset), # Sequentially
            batch_size = bs # Trains with this batch size.
        )

In [ ]:
# optimizer = AdamW([{'params': bert_model.bert.parameters(), 'weight_decay': 1e-1},
#                    {'params': bert_model.out1.parameters(), 'lr': 1e-3},
#                   #  {'params': bert_model.out1.parameters(), 'lr': 1e-3}
#                    ],
#                   lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8, # args.adam_epsilon  - default is 1e-8.
#                    )
optimizer = AdamW(params = bert_model.parameters(), lr=1e-3)
loss = nn.BCEWithLogitsLoss(reduction = 'none')

In [ ]:
epochs = 10
MAX_LEN = 512
for epoch_i in range(0, epochs):
    train_loss = [0, 0]
    for step, batch in enumerate(train_dataloader):
        if step % 100 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,} Loss {:>5,}.'.format(step, len(train_dataloader), train_loss[0]/train_loss[1]))
        inputs = batch[0]
        labels = batch[1]
        inputs_encoded = encode_sentence(inputs, MAX_LEN)
        for ele in inputs_encoded:
            inputs_encoded[ele] = inputs_encoded[ele].to(device)
        inputs_encoded['token_type_ids'] = None
        labels = labels.to(device)
        optimizer.zero_grad()
        logits = bert_model(**inputs_encoded)
        curr_batch_size = labels.size()[0]
        curr_loss = loss(logits, labels)
        curr_weights = torch.tensor(weights2).to(device)
        curr_loss = (curr_loss).sum()
        curr_loss.backward()
        train_loss[0]+= curr_loss.data.cpu().item()
        train_loss[1]+= curr_batch_size
        optimizer.step()
    print('Training Loss is : %f'%(train_loss[0]/train_loss[1]))
    bert_model.eval()
    val_loss = [0, 0]
    for step, batch in enumerate(validation_dataloader):
        inputs = batch[0]
        labels = batch[1]
        inputs_encoded = encode_sentence(inputs, MAX_LEN)
        for ele in inputs_encoded:
            inputs_encoded[ele] = inputs_encoded[ele].to(device)
        inputs_encoded['token_type_ids'] = None
        labels = labels.to(device)
        with torch.no_grad():
            logits = bert_model(**inputs_encoded)
            curr_batch_size = labels.size()[0]
            curr_loss = loss(logits, labels)
            curr_weights = torch.tensor(weights2).to(device)
            curr_loss = (curr_loss * curr_weights).sum()
            val_loss[0]+= curr_loss.data.cpu().item()
            val_loss[1]+= curr_batch_size
    bert_model.train()
    print('Validation Loss is : %f'%(val_loss[0]/val_loss[1]))

  Batch   100  of    892 Loss 2.5385136914253237.
  Batch   200  of    892 Loss 2.4822037279605866.
  Batch   300  of    892 Loss 2.458810783624649.
  Batch   400  of    892 Loss 2.444874472618103.
  Batch   500  of    892 Loss 2.4343961281776427.
  Batch   600  of    892 Loss 2.4266151334842045.
  Batch   700  of    892 Loss 2.4213785081250325.
  Batch   800  of    892 Loss 2.409520927518606.
Training Loss is : 2.403410
Validation Loss is : 3.334296
  Batch   100  of    892 Loss 2.306348143815994.
  Batch   200  of    892 Loss 2.3154092717170713.
  Batch   300  of    892 Loss 2.3119807402292887.
  Batch   400  of    892 Loss 2.3101575070619584.
  Batch   500  of    892 Loss 2.31069926071167.
  Batch   600  of    892 Loss 2.310451078414917.
  Batch   700  of    892 Loss 2.310920545033046.
  Batch   800  of    892 Loss 2.3153799030184747.
Training Loss is : 2.312337
Validation Loss is : 3.381605
  Batch   100  of    892 Loss 2.2401517045497896.
  Batch   200  of    892 Loss 2.2351161062

In [ ]:
torch.save(bert_model.state_dict(), './bert-model.pth')

In [ ]:
bert = CustomBERTModel().to(device)
bert.load_state_dict(torch.load('nlp/bert-model-base.pth'))

<All keys matched successfully>

In [ ]:
err_label = []
err_logits = []
bert_model.eval()
for i, batch in enumerate(test_dataloader):
  input = batch[0]
  labels = batch[1]
  inputs_encoded = encode_sentence(input, 512)
  curr_loss = 0
  total_batch_size = 0
  for ele in inputs_encoded:
      inputs_encoded[ele] = inputs_encoded[ele].to(device)
  labels = labels.to(device)
  with torch.no_grad():
    logits = bert_model(**inputs_encoded)
    logits_to_labels = torch.sigmoid(logits)
  err_label.extend(labels.data.cpu().numpy())
  err_logits.extend(logits_to_labels.data.cpu().numpy())
  total_batch_size += labels.size()[0]
err_logits = np.array(err_logits)
err_predictions = np.array(err_logits>0.5, dtype = np.int)
labels = np.array(err_label, dtype=np.int)
print(f1_score(labels, err_predictions, average='micro'))
print(accuracy_score(labels, err_predictions))
print(classification_report(labels, err_predictions, target_names=err_categories))

0.5367065733002033
0.5028026905829597
              precision    recall  f1-score   support

          CO       0.60      0.90      0.72       908
          GM       0.82      0.13      0.23       527
          LX       0.88      0.21      0.34       760
          OT       0.72      0.34      0.46       418

   micro avg       0.65      0.45      0.54      2613
   macro avg       0.76      0.40      0.44      2613
weighted avg       0.75      0.45      0.47      2613
 samples avg       0.55      0.54      0.54      2613



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!nvidia-smi

Tue Nov 10 04:07:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    24W /  75W |   7551MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------